In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datetime import datetime

In [2]:
import tkinter.filedialog as tkFileDialog
directory_name = "Учет контрольных"

def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')

In [3]:
file_name = open_file()
xl = None
if file_name.endswith('.xlsx'):
    xl = pd.ExcelFile(
        file_name,
        engine='openpyxl'
    )
elif file_name.endswith('.xls'):
    xl = pd.ExcelFile(
        file_name,
        engine='xlrd'
    )
print(xl.sheet_names)

xls_array = [xl.parse(sheet) for sheet in xl.sheet_names]

# for sheet in xls_array:
#     print(sheet.columns)

['Информация по студентам', 'Информация по студентам_', 'Информация по студентам__', 'Преподаватели Л, П']


In [4]:
import re
import pymorphy2
genders = {"masc": 1.0, "femn": 0.0}
morph = pymorphy2.MorphAnalyzer()

def GetGender(full_name):
    name = re.search(r"(?:.* )([^ ]*)(?:[ .]*)", full_name).group(1)
    gender = morph.parse(name)[0].tag.gender
    return genders[gender] if gender in genders.keys() else 1.0

In [12]:
# расчет для контрольных
import os

def GetInfoLose(goal):
    return "1" if float(goal)>=61 else "0"

if not os.path.exists(directory_name):
    os.makedirs(directory_name)

temp_row = ""
for sheet in xls_array:
    for name, group in sheet.groupby(["Название РМУП"]):
        if "Предмет контроля" not in group.columns:
            continue
        name_column_test = ""
        if "Контрольная работа" in set(list(group["Предмет контроля"])):
            name_column_test = "Контрольная работа"
        else:
            name_column_test = "Работа на учебной встрече"
        results = pd.DataFrame(columns=["ФИО", "Пол", "Команда", "Баллы", "Сдал(-а)"])

        name_discipline = group.head(1)["Название РМУП"].values[0]
        try:
            if (group["Название встречи"].str.contains('Контрольная ').sum()==0 and name_column_test!="Контрольная работа"):
                continue
            print(name_discipline, end=":")
            print(name_column_test, set(list(group["Предмет контроля"])))

            percent = 0
            count_presence = 0
            summ_presence = 0
            count_goals = 0
            startTime = datetime.now()
            # номер контрольной
            test_number = 1
            # # количество действий с кр (баллы и результаты до)
            # count_test_item = 0
            for index, row in group.iterrows ():
                
                if (index%(len(group)//10)==0):
                    print(f"{percent}%. Записей в итоге: {len(results)}")
                    percent+=10

                if temp_row == "" or temp_row != row["ФИО студента"]:
                    test_number = 1
                    count_presence = summ_presence = 0
                    temp_row = row["ФИО студента"]
                    results.loc[-1] = [temp_row, GetGender(temp_row), row["Команда"], row["Итог ТУ"], GetInfoLose(row["Итог ТУ"])] + [""]*(len(results.columns)-5)
                    results.index = results.index + 1  # shifting index
                
                if ("Контрольная " in row["Название встречи"] or "Контрольная работа" == row["Предмет контроля"]):
                    # if (row["Название встречи"] not in list(results.columns)):
                        # meeting_name = row["Название встречи"]
                        # if (f"Посещение до {meeting_name}" not in list(results.columns)):
                        #     results[f"Посещение до {meeting_name}"] = 0
                        #     results[f"Баллы до {meeting_name}"] = 0
                        # results[meeting_name] = 0
                        # convert_dict = {f"Посещение до {meeting_name}": float,
                        #                 f"Баллы до {meeting_name}": float,
                        #                 meeting_name: float,
                        #                 "Баллы": float
                        #             }
                        # results = results.astype(convert_dict)
                    if (f"Контрольная работа {test_number}" not in list(results.columns)):

                        meeting_name = f"Контрольная работа {test_number}"
                        if (f"Посещение до {meeting_name}" not in list(results.columns)):
                            results[f"Посещение до {meeting_name}"] = 0
                            results[f"Баллы до {meeting_name}"] = 0
                        results[meeting_name] = 0
                        convert_dict = {f"Посещение до {meeting_name}": float,
                                        f"Баллы до {meeting_name}": float,
                                        meeting_name: float,
                                        "Баллы": float
                                    }
                        results = results.astype(convert_dict)

                    if (row["Предмет контроля"] == name_column_test):
                        
                        results.loc[((results['ФИО'] == temp_row)), f"Контрольная работа {test_number}"] = row["Оценка за предметы контроля"] if row["Оценка за предметы контроля"] else 0 # row["Название встречи"]
                        
                    # if (row["Предмет контроля"] != name_column_test or name_column_test == "Контрольная работа"):
                        meeting_name = f"Контрольная работа {test_number}" #row["Название встречи"]
                        if count_presence == 0: #or results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"].values[0]!=""
                            continue
                        results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                        results.loc[((results['ФИО'] == temp_row)), f"Баллы до {meeting_name}"] = count_goals
                        if (temp_row=="Абатуров Георгий Сергеевич"):
                            print(summ_presence, count_presence, count_goals)
                        count_presence = summ_presence = count_goals = 0
                        test_number+=1


                elif (row["Предмет контроля"] != "Посещение"):
                    if not pd.isna(row["Оценка за предметы контроля"]):
                        t = row["Оценка за предметы контроля"]
                        count_goals += float(row["Оценка за предметы контроля"])
                else:
                    count_presence += 1
                    if row["Оценка за предметы контроля"]=="П":
                        summ_presence += 1
            results = results.fillna(0)
            results.drop(results.columns[-3:], axis= 1 , inplace= True ) 
            print(f"Время выполнения: {datetime.now()-startTime}")
            print("Отдельное сохранение...")
            results.to_csv(f"{directory_name}\{name_discipline}.csv", index=False, sep=';', encoding='cp1251')
            print("Сохранено")
        except Exception as e:
            print(e)      
                

Алгебра:Работа на учебной встрече {'Посещение', 'Работа на учебной встрече'}
6 11 0.5
5 5 0
8 8 12.0
0%. Записей в итоге: 94
10%. Записей в итоге: 189
20%. Записей в итоге: 285
30%. Записей в итоге: 380
40%. Записей в итоге: 475
50%. Записей в итоге: 570
60%. Записей в итоге: 666
70%. Записей в итоге: 760
80%. Записей в итоге: 856
90%. Записей в итоге: 951
Время выполнения: 0:00:05.544583
Отдельное сохранение...
Сохранено
Программирование и основы алгоритмизации 1:Контрольная работа {'Контрольная работа', 'Посещение', 'Работа на учебной встрече'}
13 14 16.0
4 4 6.0
3 7 10.0
0%. Записей в итоге: 84
10%. Записей в итоге: 180
20%. Записей в итоге: 276
30%. Записей в итоге: 372
40%. Записей в итоге: 468
50%. Записей в итоге: 564
60%. Записей в итоге: 661
70%. Записей в итоге: 756
80%. Записей в итоге: 852
90%. Записей в итоге: 948
Время выполнения: 0:00:05.183352
Отдельное сохранение...
Сохранено
Математический анализ:Контрольная работа {'Письменный ответ', 'Контрольная работа', 'Посещение

In [ ]:
# файл ПиОА.xlsx
file_name = open_file()
xl2 = None
if file_name.endswith('.xlsx'):
    xl2 = pd.ExcelFile(
        file_name,
        engine='openpyxl'
    )
elif file_name.endswith('.xls'):
    xl2 = pd.ExcelFile(
        file_name,
        engine='xlrd'
    )
print(xl2.sheet_names)

xls_array2 = [xl2.parse(sheet) for sheet in xl2.sheet_names]


# for sheet in xls_array:
#     print(sheet.columns)

In [ ]:
for sheet in xls_array2:
    # print(sheet.loc[0, :].values.flatten().tolist())
    print(sheet.columns)
    sheet.columns = sheet.loc[0, :].values.flatten().tolist()
    sheet.drop(index=[0], inplace=True)
    sheet = sheet.reset_index(drop=True)
    # sheet.dropna(subset=['Присутствие'], inplace=True)
    print(sheet.columns)

In [ ]:
data_results_presence2 = pd.DataFrame(columns=["Команда", "Баллы", "Сдал(-а)"])
print(data_results_presence2.columns)
for sheet in xls_array2:
    print(f"Записей: {len(sheet)}")
    startTime = datetime.now()
    percent = 0
    # try:
    for index, row in sheet.iterrows ():
        if (index%(len(sheet)//10)==0):
            print(f"{percent}%. Записей в итоге: {len(data_results_presence2)}")
            percent+=10
        data_results_presence2.loc[-1] = [row["Направление подготовки"], row["Итог текущ."], 1 if float(row["Итог текущ."])>=61.0 else 0] + [""]*(len(data_results_presence2.columns)-3)
        data_results_presence2.index = data_results_presence2.index + 1

        count_presence = summ_presence = count_kontr = 0

        count_kontr = 0
        for i in range(1, len(sheet.columns.to_list())):
            if sheet.columns.to_list()[i]!="Контрольная работа":
                count_presence+=1
                if (row[i]=="П"):
                    summ_presence+=1
            else:
                count_kontr+=1
                meeting_name = f"Контрольная работа {count_kontr}"
                if (f"Посещение до {meeting_name}" not in list(data_results_presence2.columns)):
                    data_results_presence2[f"Посещение до {meeting_name}"] = 0
                    data_results_presence2[meeting_name] = 0
                    convert_dict = {f"Посещение до {meeting_name}": float,
                                    meeting_name: float,
                                    "Баллы": float
                                    }
                    data_results_presence2 = data_results_presence2.astype(convert_dict)
                if count_presence == 0:
                    count_presence = summ_presence = 0
                    continue
                data_results_presence2.loc[data_results_presence2.index[-1], f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                data_results_presence2.loc[data_results_presence2.index[-1], meeting_name]  = float(row[i])

                count_presence = summ_presence = 0

    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    data_results_presence2 = data_results_presence2.fillna(0)
    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    data_results_presence2.to_csv(f"{directory_name}\Программирование и основы алгоритмизации.csv", index=False, sep=';', encoding='cp1251')
    print("Сохранено")
    # except Exception as e:
    #     print("Ошибка:", e)